In [ ]:
#Initialisation
import matplotlib.pyplot as plt
import numpy as np
# from scipy.integrate import simps
from scipy import signal as sg
from scipy.interpolate import RectBivariateSpline as ReBiSpline
# from numpy import ma
# from matplotlib import colors, ticker, cm
# from random import choice
import scipy.ndimage.filters as filters
import scipy.ndimage.morphology as morphology
import timeit
import math
import cv2
import random
from PIL import Image
import elevation_import as elev
%matplotlib inline

In [ ]:
areaName = 'Rhyd Ddu Path, Snowdon, Wales'
locSelect = [52.061386, -4.085166]
entrySelect = [[53.060900, -4.087302],
               [53.061653, -4.084131]]
areaWidth = 500

p_alpha = 1.6 # value of persistence
deg_max = 8 # maximum permissible ascent angle (30 degrees) # 2.860 = desirable, 3.732 = acceptable, 4.760 = min safety
theta_max = np.deg2rad(deg_max)

In [ ]:
elevData = elev.Elevation(areaName, locSelect[0], locSelect[1], areaWidth, entrySelect)  # Extract the elevation data at selected location
elevValue = np.transpose(elevData.elev_interpolation())  # Store the interpolated elevation values
plt.imshow(elevValue)
plt.colorbar()

In [ ]:
plt.contourf(elevValue)
# plt.contour(elevValue)
plt.colorbar()

In [ ]:
# TODO: image input?
# Read grids from image
im = Image.open("images/flat500x500.bmp")
im = im.resize((areaWidth, areaWidth), Image.ANTIALIAS)

In [ ]:
Base = np.array(im)
# Define internal quantities and variables
g_max = None
g_height = route = []
scale = 1  # m per pixel
Nx = Base[:,0,0].size  # N appears to be resolution
Ny = Base[0,:,0].size  # Nx,Ny is size, Nz is RGB level
xmin=-scale*0.5*(Nx-1)
xmax=scale*0.5*(Nx-1)
ymin=-scale*0.5*(Ny-1)
ymax=scale*0.5*(Ny-1)
x = np.linspace(xmin, xmax, Nx) # This is defining the axes and full space
y = np.linspace(ymin, ymax, Ny)
Y, X= np.meshgrid(y, x)
TrailPotential = np.zeros((Nx,Ny))
DestinationPotential=np.zeros((Nx,Ny))
Weight=np.zeros((Nx,Ny))  # Create gradient to sit on Nx, Ny
intens=np.zeros((Nx,Ny))
q_alpha=np.zeros((Nx,Ny))
expdist=np.zeros((2*Nx-1,2*Ny-1))
dest=np.zeros(2)
start=np.zeros(2)
grad=np.zeros((2,Nx,Ny))
vel=np.asarray([0.,0.])
pos=np.asarray([0.,0.])
#desdirx=ReBiSpline(x,y,grad[0,:,:],s=2)
#desdiry=ReBiSpline(x,y,grad[1,:,:],s=2)
intens[:]=0.

#parameters
t_track=50. # Track decay time - after 50 walkers ignore a trail, it decays by 1/e
dt=0.1  # dt per time step, continuous markings every dt metres
dvel=3 # desired walker velocity in m/s
tau=5.
isigma=1./2. # trail potential
conv_thresh=10.e-4
precision=1.**2 #distance to target.
eps=0.025 #random motion contribution, same for all
max_steps = 25000 # maximum amount of steps taken by walker, must be greater than 500

In [ ]:
class Slope:

    prevDir = 0 # record of previous angle of direction

    def __init__(self, posx, posy):
        try:
            xdir = 0.8*g_slope[0][posx][posy] # horizontal slope component at pos, scaled
            ydir = 0.8*g_slope[1][posx][posy] # vertical slope component at pos
        except IndexError:
            raise Exception("Walker travelled out of bounds..."
                  "\nInstruction: Decrease the walker persistence, or decrease the maximum permissible angle")
        self.xdir = xdir
        self.ydir = ydir
        self.mod = math.sqrt(xdir**2 + ydir**2)  # modulus of the gradient
        #if self.xdir == 0: # 0 exception for gradient singnularity
            # Assign angle of the gradient vector
        #    if self.ydir > 0:
        #        self.angle = math.pi/2
       #     elif self.ydir <= 0:
        #        self.angle = -math.pi/2
        #else:
        self.angle = np.arctan2(self.ydir,self.xdir)  # direction of the gradient vector  # np.arctan2() - gets result in correct quadriant
        # end __init__()

    def forbidden_angle(self, curDir):
        # (?) Still include
        # grad_max = math.tan(theta_max)
        # if self.mod < grad_max:
        #     # Ignore gradient processing if maximum gradient is not exceeded anywhere
        #     return curDir
        # else:
        # curDir = persistence(curDir) # Apply the persistence (step 7)
        # Calculate forbidden angle (alpha)
        thetaSlope = math.atan(self.mod)  # angle of the slope relative from ground to the height
        if ((self.mod or thetaSlope) == 0 ) or (thetaSlope < theta_max):
            # print("Continue (CurDir = " + str(curDir) + ") (grad = " + str(self.angle) + ") slope = ",thetaSlope)
            return curDir
        alpha = (math.pi/2 - math.asin(math.tan(theta_max)/math.tan(thetaSlope)))  # Forbidden angle from the gradient value (*)

        # Distinguish the deviation of the current direction from the closest forbidden angle and assign the new current direction accordingly
        curDir = curDir % (2*np.pi) # convert to 2pi
        # Difference between current direction and gradient, zipped around so it is always between +pi and -pi
        deviation = (curDir-self.angle + 0.5*np.pi) % np.pi - 0.5*np.pi  # deviation between pi/2 and -pi/2, absolute value
        # print('deviation = ', deviation)
        if np.abs(deviation)>alpha:
            # print('Final value unchanged \n (curDir = ' + str(curDir) + " grad = " + str(self.angle) + ") (alpha =" + str(alpha)+ ") slope = ",thetaSlope,")")
            return curDir # Current direction remains unchanged as it is outside of the forbidden zone
        if deviation < 0:
            # print("push right (CurDir =" + str(curDir) + ") (grad = " + str(self.angle) + ") (alpha =" + str(alpha)+ ") slope = ",thetaSlope,")")
            curDir = curDir-deviation-alpha  # curDir - deviation = grad (or opposite), curDir not subjected to mod
        else:
            # print("push left (CurDir =" + str(curDir) + ") (grad = " + str(self.angle) + ") (alpha =" + str(alpha)+ ") slope = ",thetaSlope,")")
            curDir = curDir-deviation+alpha

        # print(curDir)
        # Convert back to [-pi,pi]
        if (0 <= curDir <= np.pi) or (curDir > 2 * np.pi):
            curDir = (curDir % (2*np.pi))
            return curDir
        elif (np.pi < curDir < 2 * np.pi) or (curDir < 0): # Place curDir within [-pi:0]
            curDir = (curDir % (2*np.pi)) - 2 * np.pi
            return curDir
        # end forbidden_angle()

    @staticmethod
    def persistence(curDir):
        global prevDir
        # Apply persistence of direction formula (Gilks equation 6) - weighted average of the movement
        gammax= (p_alpha*np.cos(prevDir) + (1 - p_alpha)* np.cos(curDir))
        gammay= (p_alpha*np.sin(prevDir) + (1 - p_alpha)* np.sin(curDir))
        gamma = np.arctan2(gammay,gammax)  # previous direction as estimation
        return gamma

In [ ]:
class ActiveWalkerModel:

    @staticmethod
    def init():
        global z, g_max, g_nat, g_grad ,g_height, route, Base
        ##Set up map
        #Create blank arrays for map
        z = np.zeros((Nx,Ny))
        g_max=np.zeros((Nx,Ny)) # empty matrix
        g_nat=np.zeros((Nx,Ny))
        g_grad=np.zeros((Nx,Ny))
        g_nat=np.maximum(np.ones_like(g_nat),np.float64(Base[:,:,0])) # red channel, np.ones_like() sets minimum value to 1
        g_max=np.maximum(np.ones_like(g_max),np.float64(Base[:,:,1])) # green channel
        # g_height=np.maximum(np.ones_like(g_grad),np.float64(Base[:,:,2])) # blue channel
        # Assign the requested elevation data as the height matrix for the model
        g_height = np.transpose(elevData.elev_interpolation())
        # g_height=np.fromfunction(lambda i, j: 0.27*(i ), (Nx, Ny), dtype=float) # Set custom gradient
        # g_height = cv2.GaussianBlur(g_height,(5,5),2) # Apply 2D convolution using a gaussian kernel
        z=g_nat
        # Trails (start and end point) For current Map, coordinates in metres, centre of image = (0,0)
        # single possible path
        # route=np.array([[10,9.75],[-10.,-9.75]])
        # route=np.array([[24.,9.75],[-24.,-9.75]])
        # commented out for single path

        # route=np.array([[[-2.5,14.],[24.,-9.75]],
        #                 [[-2.5,14.],[24.,2.5]],
        #                 [[-2.5,14.],[-24.,9.75]],
        #                 [[24.,-9.75],[-2.75,14.]],
        #                 [[24.,-9.75],[-24.,9.75]],
        #                 [[24.,2.5],[-2.75,14.]],
        #                 [[24.,2.5],[-24.,9.75]],
        #                 [[-24.,10.],[-2.75,14.]],
        #                 [[-24.,10.],[24.,-9.75]],
        #                 [[-24.,10.],[24.,2.5]]])

    @staticmethod
    def setup_weights():
        global Weight
        #Setup weight matrix, here trapezoid rule.
        Weight[:,:]=1
        Weight[1:-1,:]=2
        Weight[:,1:-1]=2
        Weight[1:-1,1:-1]=4
        Weight*=0.25*((x[-1]-x[0])/(Nx-1))*((y[-1]-y[0])/(Ny-1))
        #0.25*((x[-1]-x[0])/(N-1))*((y[-1]-y[0])/(N-1))
        #np.exp(-np.sqrt((x[:,None]-x[N/2])**2+(y[None,:]-y[N/2])**2))*z[:,:]

    @staticmethod
    def setup_distance_matrix():
        # Setup distance matrix
        for xi in range(1,Nx+1):
            for yi in range(1,Ny+1):

                expdist[xi-1,yi-1]=np.exp(-isigma*np.sqrt((x[Nx-xi]-xmin)**2+(y[Ny-yi]-ymin)**2))
                expdist[-xi,-yi]  = expdist[xi-1,yi-1]
                expdist[-xi,yi-1] = expdist[xi-1,yi-1]
                expdist[xi-1,-yi] = expdist[xi-1,yi-1]

        # find index range > conv_thresh
        subexpdist=expdist[(expdist>conv_thresh).any(1)]
        subexpdist=subexpdist[:, np.any(subexpdist>conv_thresh, axis=0)]
        #subexpdist=subexpdist[:,np.any(subexpdist>conv_thresh, axis=0)]
        #expdist[subexpdist]=0.
        #expdist
        #subexpdist
        return subexpdist

    @staticmethod
    def calc_tr_new():
        global TrailPotential, z, Weight, subexpdist
        TrailPotential[:,:]=sg.convolve2d(z[:,:]*Weight[:,:],subexpdist[:,:],mode="same")  # 2D convolution

    @staticmethod
    def set_up_walker(route_id): # input route_id commented for one route
        #set up walker
        global vel,pos,track,intens,dest,start,route
        #  commented out as input for a single route


        # start_id = np.random.randint(0,len(elevData.entryMet))
        # for i in range(len(elevData.entryMet)+10):#create end_id that is different from start_id
        #     end_id = np.random.randint(0,len(elevData.entryMet))
        #     if end_id == start_id:
        #         continue
        #     else:
        #         break
        start_id = route_id
        end =  len(elevData.entryMet)

        r = np.concatenate((np.array(range(route_id)),np.array(range(route_id+1,end))))
        end_id = int(random.choice(r))
        #start
        start=np.array(elevData.entryMet[start_id])  # commented for simplicity
        # start = np.array(elevData.entryMet[0])  # temporary one route
        # dest = np.array(elevData.entryMet[1])  # temporary one route
        #dest=(random.choice(ends))
        dest=np.array(elevData.entryMet[end_id]) # commented for simplicity
        vel=np.array([0.,0.])
        pos=np.array(start)
        #print (pos)
        track=np.zeros((max_steps,2)) # Set up for 5000 steps maximum
        #track[0,:]=pos[:]

    @staticmethod
    def setup_potentials():
        #Calculate gradients eq 19
        #Trail gradient
        global grad,desdirx,desdiry,dest
        grad=0.003*np.array(np.gradient(TrailPotential))
        #grad=0.002*np.array(np.gradient(TrailPotential)) ORIGINAL

        #print (dest)
        #Destination potential
        DestinationPotential=-np.sqrt((dest[0]-x[:,None])**2+(dest[1]-y[None,:])**2)
        #Combine gradients
        grad+=np.array(np.gradient(DestinationPotential)[:])
        #Normalise
        #grad[:,:,:]/=(np.sqrt(grad[0,:,:]**2+grad[1,:,:]**2))
        desdirx=ReBiSpline(x,y,grad[0,:,:],s=2) # gradeint plus magnitude, Spline approximation over a rectangular mesh
        desdiry=ReBiSpline(x,y,grad[1,:,:],s=2)
        # angle array, arctans of gradient components, rebislpine
        # continuous desired angles permissible
            # 2pi periodic system aware

    @staticmethod
    def calc_path():
        global pos,vel,intens,track,dest,dvel,tau, prevDir, desdirx, desdiry
        iterator=0
        hist=10
        samp=10
        avpos=np.zeros((2,hist))
        #Setup While loop to run until either the walker reaches the destination or the walker has passed 2000 movement cycles to
        #attempt to get there
        while np.dot(pos-dest,pos-dest)>precision and iterator<max_steps: # takes 5000 steps maximum
            #set the postiion of the walker on its first then subsequent cycles
            #conditional logic saying to update the average position of the walker every 10 iterations
            #if (iterator%samp==0): avpos[:,(i%hist)//samp]=pos[:] #ORIGINAL
            if iterator%samp==0:
                avpos[:,(iterator%(hist*samp))//samp]=pos[:]
            #print((iterator%hist)//samp)
            # print(avpos)
            gradmagnitude=max(0.0001,np.sqrt(desdirx(pos[0],pos[1])**2+desdiry(pos[0],pos[1])**2))
            xi=np.array(np.random.normal(0,1,2))
            # Equation 6 in Helbing, differential in position, eliminised velocity decay components
            # gradmagnitude makes sure it is normalised, desdir not normalised
            # pos[0]+= dt *(dvel * desdirx(pos[0],pos[1])/gradmagnitude +np.sqrt(2.*eps/tau)*xi[0])  # x-position vector component
            # pos[1]+= dt *(dvel * desdiry(pos[0],pos[1])/gradmagnitude +np.sqrt(2.*eps/tau)*xi[1])  # y-position vector component
            # posGrad = math.degree(math.atan(pos[0]/pos[1]) # future position
            curDir = math.atan2(desdiry(pos[0],pos[1]),desdirx(pos[0],pos[1])) # atan2 not to flip walker around
            # pos0 = pos[0]
            # pos1 = pos[1]
            posx = int(((pos[0]-xmin)*(Nx-1))/(xmax-xmin)) # index value of position x
            posy = int(((pos[1]-ymin)*(Ny-1))/(ymax-ymin)) # index value of position y
            # print('x =' + str(posx) + '  y = ' + str(posy))
            # print('Curdir = ',curDir,' Persistent = ', persistence(curDir))
            if iterator>0: # Exception for first step
                curDir = Slope.persistence(curDir) # Apply persistence of movement
            curGradient = Slope(posx,posy)  # set the current gradient to the Slope class
            curDir = curGradient.forbidden_angle(curDir) # apply forbidden angle rule
            # print("CurDir at " + str(i) + "= " + str(curDir))
            pos[0] += dt * (dvel * math.cos(curDir))
            pos[1] += dt * (dvel * math.sin(curDir))
            # desdiry = math.sin(curDir)
            # desdirx = math.cos(curDir)
            # gradmagnitude=max(0.0001,np.sqrt(desdirx(pos[0],pos[1])**2+desdiry(pos[0],pos[1])**2))
            # xi=np.array(np.random.normal(0,1,2))
            # pos[0]+= dt *(dvel * desdirx(pos[0],pos[1])/gradmagnitude +np.sqrt(2.*eps/tau)*xi[0])  # x-position vector component
            # pos[1]+= dt *(dvel * desdiry(pos[0],pos[1])/gradmagnitude +np.sqrt(2.*eps/tau)*xi[1])  # y-position vector component
            prevDir = curDir # applying a back trace of the previous direction# Calculate current facing direction
            # pos+=dt*vel
            #vel[0]+=-1/tau*vel[0] + (dvel/tau)*desdirx(pos[0],pos[1])/gradmagnitude+np.sqrt(2.*eps/tau)*xi[0]   # Eqiation 5 in Helbing, differential in velocity
            #vel[1]+=-1/tau*vel[1] + (dvel/tau)*desdiry(pos[0],pos[1])/gradmagnitude+np.sqrt(2.*eps/tau)*xi[1]
            #Set the current position of the walker into the track array for the current iteration
            track[iterator,:]=pos[:]
            # (pos[0]-xmin)*(Nx-1)/(xmax-xmin)  - scale to resolution [Nx-1] * distance to left edge / width of whole field
            try:
                intens[int((pos[0]-xmin)*(Nx-1)/(xmax-xmin)),int((pos[1]-ymin)*(Ny-1)/(ymax-ymin))]+=1.
            except IndexError:
                raise Exception("Walker travelled out of bounds..."
                      "\nInstruction: Decrease the walker persistence, or decrease the maximum permissible angle")
            iterator+=1
            if iterator%(hist*samp)==0:
                meanpos=np.mean(avpos,axis=1)
                if np.dot(pos-meanpos,pos-meanpos)<precision:
                    print ("Stalled progress ",pos,meanpos,vel, dest)
                    break
        if iterator==(max_steps - 500): print ("Missed goal\n",dest,pos)
        return iterator

    @staticmethod
    def update_ground():
        # Calculate Q_alpha (strength of markings) eq 15
        global q_alpha,intens,z,g_max,t_track,g_nat
        q_alpha=intens*(1.-z/g_max)
        # Time evolution of ground potential
        #zdiff=(1./t_track)*(g_nat-z)+q_alpha
        z+=(1./t_track)*(g_nat-z)+q_alpha
        #cs = plt.contourf(X, Y, zdiff, cmap=cm.PuBu_r)
        #cbar = plt.colorbar()
        #plt.show
        #z[140:160,45:75]

    @staticmethod
    def detect_local_maxima(arr):
        # https://stackoverflow.com/questions/3684484/peak-detection-in-a-2d-array/3689710#3689710
        """
        Takes an array and detects the troughs using the local maximum filter.
        Returns a boolean mask of the troughs (i.e. 1 when
        the pixel's value is the neighborhood maximum, 0 otherwise)
        """
        # define an connected neighborhood
        # http://www.scipy.org/doc/api_docs/SciPy.ndimage.morphology.html#generate_binary_structure
        neighborhood = morphology.generate_binary_structure(len(arr.shape),2)
        # apply the local minimum filter; all locations of minimum value
        # in their neighborhood are set to 1
        # http://www.scipy.org/doc/api_docs/SciPy.ndimage.filters.html#minimum_filter
        local_max = (filters.maximum_filter(arr, footprint=neighborhood)==arr)
        # local_min is a mask that contains the peaks we are
        # looking for, but also the background.
        # In order to isolate the peaks we must remove the background from the mask.
        #
        # we create the mask of the background
        background = (arr==0)
        #
        # a little technicality: we must erode the background in order to
        # successfully subtract it from local_min, otherwise a line will
        # appear along the background border (artifact of the local minimum filter)
        # http://www.scipy.org/doc/api_docs/SciPy.ndimage.morphology.html#binary_erosion
        eroded_background = morphology.binary_erosion(
            background, structure=neighborhood, border_value=1)
        #
        # we obtain the final mask, containing only peaks,
        # by removing the background from the local_min mask
        detected_maxima = local_max ^ eroded_background
        return np.where(detected_maxima)

In [ ]:
class Plot:
    @staticmethod
    def path_elev():
        # plt.contourf(Y, X, g_height, levels=np.linspace(g_height.min(),g_height.max(),1000),cmap='PuBu_r')
        plt.contour(Y, X, g_height, levels=np.linspace(np.amin(g_height),np.amax(g_height),16),inline=1)
        plt.colorbar()
        plt.scatter(track[0:max_steps-1,1],track[0:max_steps-1,0],1,c='y')
        plt.show(block=False)

    @staticmethod
    def path():
        # plt.contourf(X, Y, z, levels=np.linspace(z.min(),z.max(),1000),cmap='PuBu_r')
        plt.contourf(Y, X, z, levels=np.linspace(z.min(),z.max(),1000),cmap='PuBu_r')
        plt.colorbar()
        # plt.scatter(track[0:1999,0],track[0:1999,1],1,c='y')
        plt.show(block=False)

    @staticmethod
    def directions():
        #Plot the direction
        scgrad=np.arctan2(grad[1],grad[0])
        levels = np.linspace(-np.pi, np.pi, 360)
        cs = plt.contourf(Y, X,scgrad, levels=levels,cmap='hsv')
        # cbar = plt.colorbar()
        plt.colorbar()
        plt.scatter(track[0:1999,1],track[0:1999,0])
        #plt.scatter(start, dest)
        print(start)
        print(dest)
        plt.show()

    @staticmethod
    def potentials():
        global dest
        TotPot = np.zeros((Nx,Ny))
        TotPot =- np.sqrt((dest[0]-x[:,None])**2+(dest[1]-y[None,:])**2)
        TotPot += 0.003*TrailPotential
        maxima=ActiveWalkerModel.detect_local_maxima(TotPot)
        cs = plt.contourf(Y, X, TotPot, levels=np.linspace(TotPot.min(),TotPot.max(),1000),cmap='PuBu_r')
        # cbar = plt.colorbar()
        plt.colorbar()
        print(maxima)
        plt.scatter(y[maxima[1]],x[maxima[0]])
        plt.show()
        # commit test

In [ ]:
elevData = elev.Elevation(areaName, locSelect[0], locSelect[1], areaWidth, entrySelect)  # Extract the elevation data at selected location
elevValue = elevData.elev_interpolation()  # Store the interpolated elevation values


In [ ]:
# LATER COMBINE TOGETHER
# elevValue = np.flip(elevValue,0)
elevValue = np.transpose(elevValue)
# plt.imshow(elevValue)
plt.contourf(elevValue)
plt.colorbar()
ActiveWalkerModel.init()
ActiveWalkerModel.setup_weights()
p_alpha = p_alpha/2 # scaling factor
subexpdist = ActiveWalkerModel.setup_distance_matrix()
# subexpdist.shape
timeit.timeit(ActiveWalkerModel.calc_tr_new,number=1)


In [ ]:
g_slope = np.gradient(g_height,1)
grady = g_slope[0] # vertical slope compontent
gradx = g_slope[1] # horizontal slope component
for i in range(0,50):
    ActiveWalkerModel.calc_tr_new()
    intens[:]=0.
    for j in range(0,2):
        ActiveWalkerModel.set_up_walker(np.random.randint(0,len(elevData.entryMet))) # np.random.randint(0,len(elevData.entryMet))
        ActiveWalkerModel.setup_potentials()
        # ActiveWalkerModel.calc_path()
        print(i, start," -> ", dest, pos, ActiveWalkerModel.calc_path())
    ActiveWalkerModel.update_ground()
    #plot_path()
# Last loop
ActiveWalkerModel.set_up_walker(0)
ActiveWalkerModel.setup_potentials()
# ActiveWalkerModel.calc_path()
print('last', start," -> ", dest, pos, ActiveWalkerModel.calc_path())
ActiveWalkerModel.update_ground()

print("end")


In [ ]:
elevData.map_image_get()
print("Entry = " + str(elevData.entryMet[0]) + '\n Dest'+ str(elevData.entryMet[1]))

Plot.path_elev()
Plot.path()
Plot.directions()
Plot.potentials()

In [ ]:
print('finished')
# for i in range(0,Nx):
#     for j in range(0,Ny):
#         if (np.isnan(z[i,j])):
#             print (i,j,ActiveWalkerModel.g_max[i,j],Base[i,j,0])


#Integrate z, trapezoid rule eq 20
# def calc_tr():
#    global xi,yi,TrailPotential,expdist,z,Weight,Nx,Ny
#    for xi in range(0,Nx):
#        for yi in range(0,Ny):
#            TrailPotential[xi,yi]=np.sum(expdist[Nx-1-xi:2*Nx-1-xi,Ny-1-yi:2*Ny-1-yi]*z[:,:]*Weight[:,:])

# Test additional heightS


# Defines a Plot to show the smoothing of the supplied map to represent the respective potentials of the ground, the larger
# the potentials, the more attractive the ground is to the walker

# ???
# cs = plt.contourf(X, Y, TrailPotential, levels=np.linspace(TrailPotential.min(),TrailPotential.max(),1000),cmap='PuBu_r')
# cbar = plt.colorbar()
#
# plt.scatter(track[0:1999,0],track[0:1999,1])
# plt.show()
#
#Integrate z, trapezoid rule eq 20
# def calc_tr():
#    global xi,yi,TrailPotential,expdist,z,Weight,Nx,Ny
#    for xi in range(0,Nx):
#        for yi in range(0,Ny):
#            TrailPotential[xi,yi]=np.sum(expdist[Nx-1-xi:2*Nx-1-xi,Ny-1-yi:2*Ny-1-yi]*z[:,:]*Weight[:,:])
 #%%

# #Plot the direction
# scgrad=np.arctan2(grad[1],grad[0])
# levels = np.linspace(-np.pi, np.pi, 360)
# cs = plt.contourf(X, Y,scgrad, levels=levels,cmap='hsv')

# cbar = plt.colorbar()
# # ERROR # plt.scatter(track[0:1999,0],track[0:1999,1])
# #plt.scatter(start, dest)
# print(start)
# print(dest)
# plt.show()
#

In [ ]:
# areaName = 'Rhyd Ddu Path, Snowdon, Wales'
# locSelect = [53.061386, -4.085166]
# entrySelect = [[53.061043, -4.087302],
#                [53.061793, -4.084131]]
# areaWidth = 500

# areaName = 'Slavkovsky nos, Slovakia'
# locSelect = [49.163618, 20.210788]
# entrySelect = [[49.162153, 20.216947],
#                [49.164077, 20.208601]]
# areaWidth = 1200

# areaName = 'Tatry Mountain Range, Poland'
# locSelect = [49.215265, 20.061374]
# entrySelect = [[49.215198, 20.061722],
#                [49.215256, 20.060804]]
# areaWidth = 200

# areaName = 'Wank Mountain, Germany' # - best result at 0.01, 10
# locSelect = [47.506477, 11.143055]
# entrySelect = [[47.505727, 11.142904],
#                [47.507160, 11.143308]]
# areaWidth = 300

# areaName = 'Winklmoosalm-Steinplatte, Germany'
# locSelect = [47.606918, 12.582151]
# entrySelect = [[47.608230, 12.584250],
#                [47.606674, 12.578909]]
# areaWidth = 900

# areaName = 'Penken Ski Resort, Austria'
# locSelect = [47.178091, 11.820468]
# entrySelect = [[47.178598, 11.821686],
#                [47.177239, 11.819238]]
# areaWidth = 420

# areaName = 'Mount Fuji, Japan'
# locSelect = [35.375603, 138.749592]
# entrySelect = [[35.376391, 138.751420],
#                [35.374902, 138.748095]]
# areaWidth = 500

# areaName = 'Jade Mountain, Taiwan'
# locSelect = [23.468831, 120.954830]
# entrySelect = [[23.467812, 120.954283],
#                [23.469539, 120.955888]]
# areaWidth = 900

# areaName = 'La Parva Ski Slope, Chile'
# locSelect = [-33.335387, -70.260810]
# entrySelect = [[-33.338380, -70.262335],
#                [-33.331841, -70.259477]]
# areaWidth = 900

# areaName = 'Pico El Águila, Venezuela'
# locSelect = [8.801653, -70.862748]
# entrySelect = [[8.800548, -70.861959],
#                [8.802221, -70.863895]]
# areaWidth = 900

# areaName = 'La Parva Ski Slope 2, Chile'
# locSelect = [-33.340272, -70.264660]
# entrySelect = [[-33.340780, -70.265384],
#                [-33.340278, -70.264032]]
# areaWidth = 300

# areaName = 'Base Test'
# locSelect = [0,0]
# entrySelect = [[0.0005, 0],
#                [-0.0005, 0]]
# areaWidth = 200